In [35]:

import os, sys
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(repo_root, "src"))  # so we can import from src/

import importlib
import ingestion
importlib.reload(ingestion)
import processing
importlib.reload(processing)
import features
import modeling
import evaluation

import pandas as pd
import matplotlib.pyplot as plt
from src.analysis import parse_market, find_best_odds, detect_arbitrage



In [36]:
data = ingestion.fetch_player_props(
    sport="basketball_nba",
    markets="h2h", 
    regions="us",
    odds_format="decimal"
)

odds_df = processing.flatten_odds_to_df(data, market="h2h")
print(odds_df.head())



                            game_id sport         commence_time  \
0  bbde7751a144b98ed150d7a5f7dc8f87   NBA  2025-10-21T23:30:00Z   
1  bbde7751a144b98ed150d7a5f7dc8f87   NBA  2025-10-21T23:30:00Z   
2  bbde7751a144b98ed150d7a5f7dc8f87   NBA  2025-10-21T23:30:00Z   
3  bbde7751a144b98ed150d7a5f7dc8f87   NBA  2025-10-21T23:30:00Z   
4  bbde7751a144b98ed150d7a5f7dc8f87   NBA  2025-10-21T23:30:00Z   

               home_team        away_team    bookmaker           last_update  \
0  Oklahoma City Thunder  Houston Rockets   DraftKings  2025-09-23T21:51:22Z   
1  Oklahoma City Thunder  Houston Rockets   DraftKings  2025-09-23T21:51:22Z   
2  Oklahoma City Thunder  Houston Rockets      FanDuel  2025-09-23T21:47:43Z   
3  Oklahoma City Thunder  Houston Rockets      FanDuel  2025-09-23T21:47:43Z   
4  Oklahoma City Thunder  Houston Rockets  MyBookie.ag  2025-09-23T21:50:32Z   

  market           outcome_name  price  
0    h2h        Houston Rockets   3.70  
1    h2h  Oklahoma City Thunder   

In [38]:
# calls ingestion.py function to fetch odds from The Odds API

api_key = os.getenv("ODDS_API_KEY")
market = "player_points"


odds_df = ingestion.fetch_odds(sport="basketball_nba", markets="h2h", regions="us", odds_format="decimal")
print(odds_df.head())



                            game_id         commence_time  \
0  bbde7751a144b98ed150d7a5f7dc8f87  2025-10-21T23:30:00Z   
1  bbde7751a144b98ed150d7a5f7dc8f87  2025-10-21T23:30:00Z   
2  bbde7751a144b98ed150d7a5f7dc8f87  2025-10-21T23:30:00Z   
3  bbde7751a144b98ed150d7a5f7dc8f87  2025-10-21T23:30:00Z   
4  bbde7751a144b98ed150d7a5f7dc8f87  2025-10-21T23:30:00Z   

               home_team        away_team    bookmaker market player  price  \
0  Oklahoma City Thunder  Houston Rockets   DraftKings    h2h   None   3.70   
1  Oklahoma City Thunder  Houston Rockets   DraftKings    h2h   None   1.29   
2  Oklahoma City Thunder  Houston Rockets      FanDuel    h2h   None   3.40   
3  Oklahoma City Thunder  Houston Rockets      FanDuel    h2h   None   1.34   
4  Oklahoma City Thunder  Houston Rockets  MyBookie.ag    h2h   None   3.42   

  point  
0  None  
1  None  
2  None  
3  None  
4  None  


In [43]:
# Converts raw sportsbook odds to implied probabilities, then de-vig them

processed_df = processing.american_to_prob(odds_df)
processed_df[["book", "player", "line", "price", "implied_prob", "devig_prob"]].head()


TypeError: '>' not supported between instances of 'str' and 'int'

In [40]:
# Generates rolling averages on sample player stat data

stats_df = pd.read_csv("../data/sample_player_game_logs.csv")  # makes a dummy file
feature_df = features.build_features(stats_df)
feature_df.head()


FileNotFoundError: [Errno 2] No such file or directory: '../data/sample_player_game_logs.csv'

In [ ]:
# Fits a predictive model using the feature set

X = feature_df.drop(columns=["target_points"])
y = feature_df["target_points"]

model = modeling.train_model(X, y)
preds = modeling.predict(model, X)

pd.DataFrame({"actual": y[:10], "predicted": preds[:10]})


In [ ]:
# Runs a backtest to see how the model would have performed vs. sportsbook lines

results = backtest.run_backtest(processed_df, preds, stake_size=50)
results.head()


In [ ]:
# Show profit/loss curve

plt.plot(results["date"], results["cumulative_pnl"], marker="o")
plt.title("Backtest Profit & Loss")
plt.xlabel("Date")
plt.ylabel("Cumulative PnL ($)")
plt.grid(True)
plt.show()


In [ ]:
# Example usage for arbitrage function

game = odds_json[0]  # assuming odds_json is API response list
parsed = parse_market(game, "h2h")
best_odds = find_best_odds(parsed)

arb = detect_arbitrage(best_odds)
if arb:
    print(f"✅ Arbitrage detected! Profit margin: {arb}%")
    print(best_odds)
else:
    print("No arbitrage opportunity.")
